<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/hybriderecommender_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict #

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [40]:
# CSV 파일 로드
log_df = pd.read_csv(data_path+'view_log.csv') # 로그
article_info = pd.read_csv(data_path+'article_info.csv') # 기사 정보
article_nouns_df = pd.read_csv(data_path+'article_nouns.csv', index_col= 0) # 기사 내용이 전처리된 데이터

In [41]:
article_nouns_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3008 entries, 0 to 3007
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   articleID      3008 non-null   object
 1   Title          3008 non-null   object
 2   Content        3008 non-null   object
 3   Format         3008 non-null   object
 4   Language       3008 non-null   object
 5   userID         3008 non-null   object
 6   userCountry    659 non-null    object
 7   userRegion     657 non-null    object
 8   Content_Nouns  3008 non-null   object
dtypes: object(9)
memory usage: 235.0+ KB


In [42]:
article_nouns_df

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion,Content_Nouns
0,ARTICLE_0000,19 Tips For Everyday Git Use,using git full time past year wanted share pra...,HTML,en,USER_0683,NaN,NaN,git time year share tip way git git cheat shee...
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,intel acquired computer vision machine learnin...,HTML,en,USER_1129,NaN,NaN,computer vision machine learning startup itsee...
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,one reason angularjs great work developed arou...,HTML,en,USER_0256,NaN,NaN,reason work idea framework check source core t...
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,despite recent positive news renewed interest ...,HTML,en,USER_1304,NaN,NaN,news interest investor country tsumoney crude ...
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,last year around time wrote big reverse web wo...,HTML,en,USER_0336,NaN,NaN,year time web architecture web information per...
...,...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,iniciei jornada swift saindo versão muuuuuitos...,HTML,pt,USER_0882,BR,MG,iniciei versão problemas vez linguagem desenvo...
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,amazing journey get happened without feedback ...,HTML,en,USER_0220,NaN,NaN,journey feedback community member contribution...
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,part one series post kotlin click part discus ...,HTML,en,USER_1010,BR,SP,part series post part discus design language d...
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,instituto tecnológico aeronáutica ita fundado ...,HTML,pt,USER_1210,NaN,NaN,instituto cursos plataforma plataforma usuário...


In [43]:
# 불필요한 열 제거
article_nouns_df = article_nouns_df.drop(['userCountry', 'userRegion', 'userID'], axis=1)

# 데이터 병합
merged_df = pd.merge(log_df, article_nouns_df, on='articleID')

# 기사 데이터 중 로그 데이터에 없는 기사 제거
article_nouns_df = article_nouns_df[article_nouns_df['articleID'].isin(merged_df['articleID'].unique())]

# 데이터 병합
merged_df = pd.merge(log_df, article_nouns_df, on='articleID')

In [83]:
article_nouns_df # 128개 기사 제거 (사용자와 상호작용이 없는 기사)

,articleID,Title,Content,Format,Language,Content_Nouns
0,0,19 Tips For Everyday Git Use,using git full time past year wanted share pra...,HTML,en,git time year share tip way git git cheat shee...
1,1,Intel buys computer vision startup Itseez to i...,intel acquired computer vision machine learnin...,HTML,en,computer vision machine learning startup itsee...
2,2,Practical End-to-End Testing with Protractor,one reason angularjs great work developed arou...,HTML,en,reason work idea framework check source core t...
3,3,Corporate venture growth in Brazil is another ...,despite recent positive news renewed interest ...,HTML,en,news interest investor country tsumoney crude ...
4,4,Cross-channel user experiences with Drupal (aw...,last year around time wrote big reverse web wo...,HTML,en,year time web architecture web information per...
...,...,...,...,...,...,...
3003,2874,Como consumir conteúdo de qualidade em iOS - C...,iniciei jornada swift saindo versão muuuuuitos...,HTML,pt,iniciei versão problemas vez linguagem desenvo...
3004,2875,Aurelia 1.0 is Here!!!,amazing journey get happened without feedback ...,HTML,en,journey feedback community member contribution...
3005,2876,Lessons from converting an app to 100% Kotlin ...,part one series post kotlin click part discus ...,HTML,en,part series post part discus design language d...
3006,2877,ITA está oferecendo 10 cursos gratuitos a dist...,instituto tecnológico aeronáutica ita fundado ...,HTML,pt,instituto cursos plataforma plataforma usuário...


In [44]:
merged_df

,userID,articleID,userRegion,userCountry,Title,Content,Format,Language,Content_Nouns
0,USER_0000,ARTICLE_0661,NY,US,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
1,USER_0067,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
2,USER_0414,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
3,USER_0557,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
4,USER_1163,ARTICLE_0661,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
...,...,...,...,...,...,...,...,...,...
42712,USER_1420,ARTICLE_0714,SP,BR,Sponsored: Five reasons why retailers should o...,unsupported browser better experience please u...,HTML,en,browser experience browser version browser exp...
42713,USER_1420,ARTICLE_1711,SP,BR,Automação de Marketing: o seu negócio está pro...,automação marketing ai todos definitivamente p...,HTML,pt,automação jogo automação transformação process...
42714,USER_1420,ARTICLE_1732,SP,BR,Digital Tool as Strategic Enabler for Banking ...,brian pallas founder ceo opportunity network a...,HTML,en,opportunity network director communication inv...
42715,USER_1420,ARTICLE_1732,SP,BR,Digital Tool as Strategic Enabler for Banking ...,brian pallas founder ceo opportunity network a...,HTML,en,opportunity network director communication inv...


### 협업 필터링 점수 계산 (baseline)

In [45]:
# 사용자-기사 행렬 생성
user_article_matrix = merged_df.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_article_matrix.values)

# 협업 필터링 추천 점수 계산
user_based_scores = user_similarity.dot(user_article_matrix.values) / np.array([np.abs(user_similarity).sum(axis=1)]).T

### 컨텐츠 기반 필터링 점수 계산을 위한 칼럼 벡터화

In [46]:
# 라벨 인코더 생성 및 학습
article_label_encoder = LabelEncoder()
merged_df['articleID'] = article_label_encoder.fit_transform(merged_df['articleID'])
article_nouns_df['articleID'] = article_label_encoder.transform(article_nouns_df['articleID'])

# 유저 아이디 라벨 인코딩 적용
user_label_encoder = LabelEncoder()
merged_df['userID'] = user_label_encoder.fit_transform(merged_df['userID'])

In [47]:
merged_df

,userID,articleID,userRegion,userCountry,Title,Content,Format,Language,Content_Nouns
0,0,632,NY,US,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
1,67,632,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
2,413,632,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
3,553,632,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
4,1157,632,SP,BR,Facebook says its new AI is almost as good as ...,facebook developed new ai called deeptext clai...,HTML,en,deeptext claim people accuracy time language d...
...,...,...,...,...,...,...,...,...,...
42712,1414,683,SP,BR,Sponsored: Five reasons why retailers should o...,unsupported browser better experience please u...,HTML,en,browser experience browser version browser exp...
42713,1414,1634,SP,BR,Automação de Marketing: o seu negócio está pro...,automação marketing ai todos definitivamente p...,HTML,pt,automação jogo automação transformação process...
42714,1414,1654,SP,BR,Digital Tool as Strategic Enabler for Banking ...,brian pallas founder ceo opportunity network a...,HTML,en,opportunity network director communication inv...
42715,1414,1654,SP,BR,Digital Tool as Strategic Enabler for Banking ...,brian pallas founder ceo opportunity network a...,HTML,en,opportunity network director communication inv...


In [48]:
article_nouns_df

,articleID,Title,Content,Format,Language,Content_Nouns
0,0,19 Tips For Everyday Git Use,using git full time past year wanted share pra...,HTML,en,git time year share tip way git git cheat shee...
1,1,Intel buys computer vision startup Itseez to i...,intel acquired computer vision machine learnin...,HTML,en,computer vision machine learning startup itsee...
2,2,Practical End-to-End Testing with Protractor,one reason angularjs great work developed arou...,HTML,en,reason work idea framework check source core t...
3,3,Corporate venture growth in Brazil is another ...,despite recent positive news renewed interest ...,HTML,en,news interest investor country tsumoney crude ...
4,4,Cross-channel user experiences with Drupal (aw...,last year around time wrote big reverse web wo...,HTML,en,year time web architecture web information per...
...,...,...,...,...,...,...
3003,2874,Como consumir conteúdo de qualidade em iOS - C...,iniciei jornada swift saindo versão muuuuuitos...,HTML,pt,iniciei versão problemas vez linguagem desenvo...
3004,2875,Aurelia 1.0 is Here!!!,amazing journey get happened without feedback ...,HTML,en,journey feedback community member contribution...
3005,2876,Lessons from converting an app to 100% Kotlin ...,part one series post kotlin click part discus ...,HTML,en,part series post part discus design language d...
3006,2877,ITA está oferecendo 10 cursos gratuitos a dist...,instituto tecnológico aeronáutica ita fundado ...,HTML,pt,instituto cursos plataforma plataforma usuário...


In [49]:
# merged_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (지역, 나라, 언어, 형식)
merged_df_categories = merged_df[['userRegion', 'userCountry', 'Format', 'Language']]

# One-Hot Encoding 수행
merged_df_encoded_categories = encoder.fit_transform(merged_df_categories).toarray()

#### 원본 content칼럼의 len 벡터화

In [50]:
# Content 컬럼의 길이 계산
merged_df['Content_length'] = merged_df['Content'].apply(len)

# Content 길이를 특징으로 추가
content_lengths = merged_df['Content_length'].values.reshape(-1, 1)

#### title + content 칼럼의 벡터화

In [51]:
# 기사 : 제목 + 본문 벡터화
tfidf = TfidfVectorizer(
    max_features=10000, #단어 사전에 포함할 최대 단어 수, 빈도수가 높은 단어부터 선택됨
    ngram_range=(1, 2), # n-gram의 범위 지정, unigram과 bigram모두 사용
    max_df=0.8, #문서의 80% 이상 나타나는 단어 무시
    min_df=3, #최소 3회 이상 나타나는 단어 사용
    sublinear_tf=True,
    smooth_idf=True
)
tfidf_matrix = tfidf.fit_transform(merged_df['Title'] + ' ' + merged_df['Content_Nouns'])

In [52]:
# TF-IDF 행렬과 원핫 인코딩된 피처 벡터 결합
combined_features = np.hstack((tfidf_matrix.toarray(), merged_df_encoded_categories, content_lengths))

### 유저-기사 리스트 생성

In [53]:
# 유저별 기사 읽기 패턴 추출
user_read_articles = defaultdict(list) # 빈 사전 리스트 생성
for _, row in merged_df.iterrows():
    user_read_articles[row['userID']].append(row['articleID'])

In [69]:
user_read_articles

defaultdict(list,
            {0: [632,
              2212,
              2212,
              1282,
              1282,
              1046,
              1417,
              991,
              991,
              2156,
              2156,
              2156,
              2156,
              1204,
              1204,
              85,
              898,
              2185,
              1691,
              846,
              1146,
              2713,
              2713,
              390,
              390,
              390,
              2332,
              701,
              1547,
              2642,
              1727,
              1652,
              1176,
              2864,
              2688],
             67: [632,
              991,
              1691,
              390,
              2688,
              2746,
              2746,
              2743,
              1825,
              2666,
              1816,
              872,
              1043,
              2005,
         

### 하이브리드 추천 점수 계산 함수

In [56]:
def recommend_articles(userID, top_n=5, content_weight=0.3, collaborative_weight=0.7):

    # 사용자가 아무 기사를 읽지 않았거나 유효한 사용자 ID가 아닐 경우, 무작위로 top_n개의 기사를 반환
    if userID not in user_read_articles or not user_read_articles[userID]:
        return article_nouns_df.sample(n=top_n)[['Title', 'Content_Nouns']]

    # 사용자가 읽은 기사의 피처 벡터 평균 계산
    read_articles = user_read_articles[userID]
    read_articles_features = combined_features[read_articles]

    # 사용자가 읽은 기사가 없을 경우, 무작위로 top_n개의 기사를 반환
    if read_articles_features.shape[0] == 0:
        return article_nouns_df.sample(n=top_n)[['Title', 'Content_Nouns']]

    # 사용자가 읽은 기사들의 피처 벡터 평균 계산
    avg_features = np.mean(read_articles_features, axis=0).reshape(1, -1)

    # 사용자가 읽은 기사들의 피처 벡터 평균과 모든 기사들의 피처 벡터 간의 코사인 유사도 계산
    content_similarities = cosine_similarity(avg_features, combined_features).flatten()

    # 사용자 기반 협업 필터링 점수
    collaborative_similarities = user_based_scores[userID]

    # 하이브리드 추천 점수 계산 (콘텐츠 기반 필터링과 협업 필터링의 가중합)
    hybrid_similarities = np.zeros(collaborative_similarities.shape) # 배열 초기화 (점수 저장)

    for i, article_id in enumerate(article_nouns_df['articleID']):
        if article_id in user_read_articles[userID]: # 사용자가 이미 읽은 기사에 대해서는 콘텐츠 기반 필터링 점수와 협업 필터링 점수의 가중합을 계산
            idx = article_nouns_df[article_nouns_df['articleID'] == article_id].index[0]
            hybrid_similarities[i] = (content_weight * content_similarities[idx] +
                                      collaborative_weight * collaborative_similarities[i])

        else: # 사용자가 읽지 않은 기사에 대해서는 협업 필터링 점수만 사용 (유사한 사용자에 행동에 기반하여 개인화된 추천)
            hybrid_similarities[i] = collaborative_similarities[i]

    # 유사도가 높은 상위 top_n 개의 기사 인덱스 추출
    similar_articles_indices = hybrid_similarities.argsort()[-top_n:][::-1]

    # 추천 기사 반환
    recommendations = article_nouns_df.iloc[similar_articles_indices]
    return recommendations

In [57]:
recommend_articles(0)

,articleID,Title,Content,Format,Language,Content_Nouns
411,390,Embracing Agile,idea brief problem agile method scrum kanban l...,HTML,en,idea problem method development function compa...
1230,1176,You won't recognize the new world of digital p...,bi intelligence modern smartphone remarkable d...,HTML,en,intelligence device device pocket task camera ...
2255,2156,Equal Pay Day in the spotlight this year,heard surely heard woman earn le money men oft...,HTML,en,woman money men woman cent dollar men income a...
1621,1547,Innovation is in all the wrong places,live pretty cosmopolitan futuristic life atop ...,HTML,en,life glass skyscraper pizza drone order taxi a...
2806,2688,Time to Re-Think Design Thinking,olof schybergson ceo shelley evenson head orga...,HTML,en,evolution fjord design innovation accenture co...


In [59]:
# 사용자 ID별 기사 추천
user_recommendations = []

for encoded_user_id in merged_df['userID'].unique():
    # 추천 기사 인덱스 가져오기
    recommended_articles = recommend_articles(encoded_user_id)

    # 원래 유저 아이디로 역변환
    original_user_id = user_label_encoder.inverse_transform([encoded_user_id])[0]

    for idx in recommended_articles.index:
        # 원래 기사 아이디로 역변환
        original_article_id = article_label_encoder.inverse_transform([article_nouns_df.loc[idx, 'articleID']])[0]
        # 추천 결과를 리스트에 저장
        user_recommendations.append({
            'userID': original_user_id,
            'articleID': original_article_id
        })

In [60]:
recommendations_df = pd.DataFrame(user_recommendations)

# sample_submission.csv 형태로 DataFrame 생성
recommendations_df = recommendations_df.sort_values(by ='userID').reset_index(drop=True)

In [61]:
recommendations_df

,userID,articleID
0,USER_0000,ARTICLE_0411
1,USER_0000,ARTICLE_1230
2,USER_0000,ARTICLE_2255
3,USER_0000,ARTICLE_1621
4,USER_0000,ARTICLE_2806
...,...,...
7070,USER_1420,ARTICLE_0614
7071,USER_1420,ARTICLE_1901
7072,USER_1420,ARTICLE_0714
7073,USER_1420,ARTICLE_2865


In [ ]:
recommendations_df.to_csv(data_path + 'hybrid_recommendations_sh_collabo.csv', index=False)